In [1]:
from utils import (
    get_mnist_data_loaders,
    NN_FC_CrossEntropy,
    compute_validation_accuracy_multi,
    train_one_epoch,
)

from fastprogress.fastprogress import master_bar

import torch

import matplotlib.pyplot as plt
from jupyterthemes import jtplot

jtplot.style(context="talk")

In [2]:
# Configuration parameters
data_path = "../data"
seed = 0
torch.manual_seed(seed)

# Hyperparameters
num_epochs = 4
batch_size = 128
valid_batch_size = 0
# The optimizer includes default hyperparameter values

# Training device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using '{device}' device.")

Using 'cuda' device.


In [3]:
# Get data loaders
train_loader, valid_loader = get_mnist_data_loaders(
    data_path, batch_size, valid_batch_size
)

/opt/mambaforge/envs/cs152/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-pma2oi4d/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
# Create neural network model
nx = train_loader.dataset.data.shape[1:].numel()
ny = len(train_loader.dataset.classes)
layer_sizes = (nx, 20, 20, ny)

model = NN_FC_CrossEntropy(layer_sizes).to(device)

# Training utilities
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [17]:
def init_weights(layer):
    if type(layer) == torch.nn.Linear:
        print("Initializing", layer)
#         layer.weight.data.fill_(0.0)
#         layer.bias.data.fill_(0.0)

#         layer.weight.data.fill_(1.0)
#         layer.bias.data.fill_(1.0)

#         layer.weight.data.uniform_()
#         layer.bias.data.fill_(0.0)
        
#         layer.weight.data.normal_()
#         layer.bias.data.fill_(0.0)

        # torch.nn.init.xavier_uniform_(layer.weight)
        torch.nn.init.kaiming_normal_(layer.weight)


model.apply(init_weights)

Initializing Linear(in_features=784, out_features=20, bias=True)
Initializing Linear(in_features=20, out_features=20, bias=True)
Initializing Linear(in_features=20, out_features=10, bias=True)


NN_FC_CrossEntropy(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Sequential(
      (0): Linear(in_features=784, out_features=20, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=20, out_features=20, bias=True)
      (1): ReLU()
    )
    (3): Linear(in_features=20, out_features=10, bias=True)
  )
)

In [18]:
# Training loop
mb = master_bar(range(num_epochs))
compute_validation_accuracy_multi(valid_loader, model, criterion, device, mb, 0)
for epoch in mb:
    train_one_epoch(train_loader, model, criterion, optimizer, device, mb)
    loss, accuracy = compute_validation_accuracy_multi(
        valid_loader, model, criterion, device, mb, epoch + 1
    )